In [1]:
import numpy as np
import os
import itertools
from numpy.linalg import eig
from sklearn.model_selection import train_test_split


# Helper function to compute risk on pairs
def compute_risk_pairs(diffTe, pYTe, b, M):
    slack = np.maximum(0, 1 + pYTe.T * (np.sum((M @ diffTe.T) * diffTe.T, axis=0) - b))
    return np.mean(slack)


# SGD algorithm for metric learning with hinge loss objective
def sgd_pairs(
    X, Y, diffVal, pYVal, diffTe, pYTe, b, mode, n2, eta0, maxIter, iterRisk, saveDir
):
    n, d = X.shape
    M = np.eye(d)  # Initialize M to identity matrix
    countRisk = 0

    # Number of pairs in mini-batch
    m = int(n2 * (n2 - 1) / 2)

    res = []

    for k in range(1, maxIter + 1):
        if mode == "p":
            pairs = np.zeros((m, 2), dtype=int)
            pairs[:, 0] = np.random.choice(n, m, replace=True)
            pairs[:, 1] = np.random.choice(n, m, replace=True)
        elif mode == "i":
            s = np.random.choice(n, n2, replace=True)
            pairs = np.array(list(itertools.combinations(s, 2)))
        else:
            raise ValueError("Invalid mode")

        pY = np.ones(m) * -1
        pY[Y[pairs[:, 0]] == Y[pairs[:, 1]]] = 1

        gradM = np.zeros((d, d))
        diff = X[pairs[:, 0], :] - X[pairs[:, 1], :]
        slack = (1 + pY * (np.sum((M @ diff.T) * diff.T, axis=0) - b)) > 0
        for i in range(m):
            if slack[i]:
                gradM += pY[i] * np.outer(diff[i], diff[i])
        gradM /= m

        M -= eta0 * (1 / k) * gradM

        if k in iterRisk:
            # Project onto the PSD cone
            V, L = eig(M)
            V = np.real(V)
            L = np.real(L)
            ind = np.where(np.diag(L) > 0)
            M2 = V[:, ind] @ np.diag(L[ind]) @ V[:, ind].T

            # Compute val risk
            RVal = compute_risk_pairs(diffVal, pYVal, b, M2)
            # Compute test risk
            RTe = compute_risk_pairs(diffTe, pYTe, b, M2)

            res.append([k, RVal, RTe])
            print(f"iter {k}, val risk {RVal}, test risk {RTe}")
            np.save(os.path.join(saveDir, f"M_{k}.npy"), M2)

    np.save(os.path.join(saveDir, "res.npy"), res)


# Main MNIST function
def mnist(b, mode, n2, eta0, maxIter, valSize, testSize, seed):
    b = float(b)
    n2 = int(n2)
    eta0 = float(eta0)
    maxIter = int(maxIter)
    valSize = int(valSize)
    testSize = int(testSize)
    seed = int(seed)

    iterRisk = (
        list(range(1, 101, 10))
        + list(range(100, 1001, 100))
        + list(range(1000, 10001, 1000))
    )

    # Load data (adjust path if necessary)
    # data = np.load("/cluster/fhgfs/abellet/MNIST/data/mnist_train.npy")
    data = np.load("data/data_50000_10000_10000.npz")
    XTr = data["X_train"]
    YTr = data["y_train"]
    XTe = data["X_test"]
    YTe = data["y_test"]

    # Set seed for reproducibility
    np.random.seed(11111)

    # Generate validation pairs
    nTrain = XTr.shape[0]
    pairsVal = np.zeros((valSize, 2), dtype=int)
    pairsVal[:, 0] = np.random.choice(nTrain, valSize, replace=True)
    pairsVal[:, 1] = np.random.choice(nTrain, valSize, replace=True)
    pYVal = np.ones(valSize) * -1
    pYVal[YTr[pairsVal[:, 0]] == YTr[pairsVal[:, 1]]] = 1
    diffVal = XTr[pairsVal[:, 0], :] - XTr[pairsVal[:, 1], :]

    # Generate test pairs
    nTest = XTe.shape[0]
    pairsTest = np.zeros((testSize, 2), dtype=int)
    pairsTest[:, 0] = np.random.choice(nTest, testSize, replace=True)
    pairsTest[:, 1] = np.random.choice(nTest, testSize, replace=True)
    pYTe = np.ones(testSize) * -1
    pYTe[YTe[pairsTest[:, 0]] == YTe[pairsTest[:, 1]]] = 1
    diffTe = XTe[pairsTest[:, 0], :] - XTe[pairsTest[:, 1], :]

    # Create save directory
    saveDir = f"bias{b}_mode{mode}_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}.{seed}"
    os.makedirs(saveDir, exist_ok=True)

    # Set random seed for algorithm
    if mode == "p":
        np.random.seed(seed)
    elif mode == "i":
        np.random.seed(seed + 1000)
    else:
        raise ValueError("Invalid mode")

    # Run the algorithm
    sgd_pairs(
        XTr,
        YTr,
        diffVal,
        pYVal,
        diffTe,
        pYTe,
        b,
        mode,
        n2,
        eta0,
        maxIter,
        iterRisk,
        saveDir,
    )

In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt


# Function to retrieve MNIST experiment results
def make_res2(b, n2, eta0, maxIter, valSize, testSize, runs):
    base_path = f"res_MNIST2/bias{b}_modei_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}.1/res.npy"
    res = np.load(base_path, allow_pickle=True)
    resITr = res[:, :2]
    resITe = res[:, [0, 2]]

    for s in range(2, runs + 1):
        path = f"res_MNIST2/bias{b}_modei_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}.{s}/res.npy"
        res = np.load(path, allow_pickle=True)
        resITr = np.column_stack((resITr, res[:, 1]))
        resITe = np.column_stack((resITe, res[:, 2]))

    base_path = f"res_MNIST2/bias{b}_modep_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}.1/res.npy"
    res = np.load(base_path, allow_pickle=True)
    resPTr = res[:, :2]
    resPTe = res[:, [0, 2]]

    for s in range(2, runs + 1):
        path = f"res_MNIST2/bias{b}_modep_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}.{s}/res.npy"
        res = np.load(path, allow_pickle=True)
        resPTr = np.column_stack((resPTr, res[:, 1]))
        resPTe = np.column_stack((resPTe, res[:, 2]))

    # Compute means and standard deviations
    meanITr = np.mean(resITr[:, 1 : runs + 1], axis=1)
    meanITe = np.mean(resITe[:, 1 : runs + 1], axis=1)
    meanPTr = np.mean(resPTr[:, 1 : runs + 1], axis=1)
    meanPTe = np.mean(resPTe[:, 1 : runs + 1], axis=1)
    stdITr = np.std(resITr[:, 1 : runs + 1], axis=1, ddof=1)
    stdITe = np.std(resITe[:, 1 : runs + 1], axis=1, ddof=1)
    stdPTr = np.std(resPTr[:, 1 : runs + 1], axis=1, ddof=1)
    stdPTe = np.std(resPTe[:, 1 : runs + 1], axis=1, ddof=1)

    # Plot training results
    plt.plot(resPTr[:, 0], meanPTr, "b", linewidth=3)
    plt.fill_between(
        resPTr[:, 0], meanPTr - stdPTr, meanPTr + stdPTr, color="blue", alpha=0.3
    )
    plt.plot(resITr[:, 0], meanITr, "r", linewidth=3)
    plt.fill_between(
        resITr[:, 0], meanITr - stdITr, meanITr + stdITr, color="red", alpha=0.3
    )

    save_path = f"res_MNIST2/plot_bias{b}_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_runs{runs}_meanstdTr.eps"
    plt.savefig(save_path, format="eps")
    plt.clf()

    # Plot test results
    plt.plot(resPTe[:, 0], meanPTe, "b", linewidth=3)
    plt.fill_between(
        resPTe[:, 0], meanPTe - stdPTe, meanPTe + stdPTe, color="blue", alpha=0.3
    )
    plt.plot(resITe[:, 0], meanITe, "r", linewidth=3)
    plt.fill_between(
        resITe[:, 0], meanITe - stdITe, meanITe + stdITe, color="red", alpha=0.3
    )

    save_path = f"res_MNIST2/plot_bias{b}_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_runs{runs}_meanstdTe.eps"
    plt.savefig(save_path, format="eps")
    plt.clf()

    # Plot individual training results
    plt.figure()
    for s in range(1, runs + 1):
        plt.plot(resPTr[:, 0], resPTr[:, s], "b", linewidth=1)
        plt.plot(resITr[:, 0], resITr[:, s], "r", linewidth=1)

    save_path = f"res_MNIST2/plot_bias{b}_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_runs{runs}_indTr.eps"
    plt.savefig(save_path, format="eps")
    plt.clf()

    # Plot individual test results
    plt.figure()
    for s in range(1, runs + 1):
        plt.plot(resPTe[:, 0], resPTe[:, s], "b", linewidth=1)
        plt.plot(resITe[:, 0], resITe[:, s], "r", linewidth=1)

    save_path = f"res_MNIST2/plot_bias{b}_n2{n2}_eta0{eta0}_maxIter{maxIter}_valSize{valSize}_testSize{testSize}_runs{runs}_indTe.eps"
    plt.savefig(save_path, format="eps")
    plt.clf()

In [3]:
b = 2
n2 = 23
eta0 = 0.01
maxIter = 5000
valSize = 1000
testSize = 1000
runs = 5

In [4]:
mode = "i"
seed = 1

In [5]:
mnist(b, mode, n2, eta0, maxIter, valSize, testSize, seed)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [5]:
make_res2(b=2, n2=23, eta0=0.01, maxIter=5000, valSize=1000, testSize=1000, runs=5)

FileNotFoundError: [Errno 2] No such file or directory: 'res_MNIST2/bias2_modei_n223_eta00.01_maxIter5000_valSize1000_testSize1000.1/res.npy'